In [ ]:
import os
import sys
import glob
import warnings
from copy import deepcopy

PROJECT_DIR = '/Users/research/projects/kepler-ecc-rp/'
sys.path.append(PROJECT_DIR)

In [ ]:
import astropy.constants as apc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy.interpolate import CubicSpline
from scipy import stats
from scipy.stats import gaussian_kde, ks_2samp

from utils.astro import calc_T14_circ, jacobian, detection_prior, duration_ratio
from utils.eccsamp import imp_sample_rhostar
from utils.io import load_dr25_data_from_hdf5
from utils.stats import weighted_percentile, draw_random_samples

pi = np.pi

RSAU = (apc.R_sun/apc.au).value                                 # solar radius [AU]
RSRE = (apc.R_sun/apc.R_earth).value                            # R_sun/R_earth
RHOSUN_GCM3 = (3*apc.M_sun/(4*pi*apc.R_sun**3)).value/1000      # solar density [g/cm^3]

## Load DR25 Catalog

In [ ]:
MULTIPLICITY = (1,99)
PER_LIM      = (1,100)
RAD_TYPE     = 'rp'
RAD_LIM      = (0.5,16.0)
MSTAR_LIM    = (0.,10.)
RSTAR_LIM    = (0.7,1.4)
FEH_LIM      = (-0.5,0.5)
TEFF_LIM     = (4700,6500)
AGE_LIM      = (0,14)
E_DETPRIOR   = True
B_DETPRIOR   = False

In [ ]:
DR25_CATALOG = os.path.join(PROJECT_DIR, 'Catalogs/kepler_dr25_gaia_dr2_crossmatch.csv')
catalog = pd.read_csv(DR25_CATALOG, index_col=0)

# hard-code period and radius limits
use  = (catalog.period > 1) * (catalog.period < 300)
use *= (catalog.rp > 0) * (catalog.rp < 16)

# remove likely false positives
use *= (catalog.fpp < 0.1) + (catalog.disposition == 'CONFIRMED')

# clean up stellar sample
use *= catalog.logg > 4.0                                 # surface gravity as proxy for main sequence
use *= (catalog.rcf - 1 < 0.05) + np.isnan(catalog.rcf)   # radius correction factor (Furlan+ 2017)
use *= catalog.ruwe < 1.4                                 # Gaia RUWE

# slice subpopulation
use *= ((catalog.period >= PER_LIM[0]) &
        (catalog.period <= PER_LIM[1]) &
        (catalog.mstar >= MSTAR_LIM[0]) &
        (catalog.mstar <= MSTAR_LIM[1]) &
        (catalog.rstar >= RSTAR_LIM[0]) &
        (catalog.rstar <= RSTAR_LIM[1]) &
        (catalog.feh >= FEH_LIM[0]) &
        (catalog.feh <= FEH_LIM[1]) &
        (catalog.teff >= TEFF_LIM[0]) &
        (catalog.teff <= TEFF_LIM[1]) &
        (catalog.age >= AGE_LIM[0]) &
        (catalog.age <= AGE_LIM[1])
       )

# update targets and catalog
catalog = catalog.loc[use].reset_index(drop=True)
targets = np.array(catalog.planet_name)

## Trend comparison

In [ ]:
occurrence = pd.read_csv(os.path.join(PROJECT_DIR, "Results/fulton-petigura-2018-occurrence.csv"))
ecc_data  = pd.read_csv(os.path.join(PROJECT_DIR, "Results/ecc-rp-singles-empirical.csv"))
ecc_model = pd.read_csv(os.path.join(PROJECT_DIR, "Results/ecc-rp-singles-smooth-model-empirical.csv"))

In [ ]:
Rp = np.array([0.5, 1, 2, 4, 8, 16])

sns.set_context('paper', font_scale=1.1)

fig, ax = plt.subplots(3,1, figsize=(8,8))

# occurrence rates
n = 7

lnR = np.log(occurrence.bin_center.values)
occ = np.array(occurrence.occ.values, dtype='float')
occ_err = np.array(occurrence.occ_err.values, dtype='float')
occ_err[occ_err==100] = 0

ax[0].step(lnR[n:], occ[n:], where='mid', color='k', lw=2)
ax[0].step(lnR[:n+1], occ[:n+1], where='mid', color='lightgrey', ls='--', lw=2)
ax[0].errorbar(lnR[n:], occ[n:], yerr=occ_err[n:], fmt='none', capsize=5, color='k')
ax[0].axvline(np.log(3.5), color='C3', ls=':', lw=2, zorder=0)
ax[0].set_xlim(np.log(0.5), np.log(16))
ax[0].set_xticks(np.log(Rp), Rp)
ax[0].set_yticks([0, 0.04, 0.08, 0.12], [0,4,8,16])
ax[0].set_ylabel("Planets per\n100 stars",fontsize=14)


# metallicity
ax[1].scatter(np.log(catalog.rp), catalog.feh, s=10, facecolors='none', edgecolors='k', lw=0.5)
ax[1].axvline(np.log(3.5), color='C3', ls=':', lw=2, zorder=0)
ax[1].set_xlim(np.log(0.5), np.log(16))
ax[1].set_xticks(np.log(Rp), Rp)
ax[1].set_ylim(-0.63, 0.63)
ax[1].set_ylabel("Metallicity", fontsize=14)


# eccentricity
lnR = np.log(ecc_data.rp)
ecc = [ecc_data.ecc_16, ecc_data.ecc_50, ecc_data.ecc_84]

ax[2].errorbar(lnR, ecc[1], yerr=(ecc[1]-ecc[0], ecc[2]-ecc[1]), fmt='ko')
ax[2].plot(ecc_model.xpred, ecc_model.ypred_50, color='grey', lw=2)
ax[2].fill_between(ecc_model.xpred, ecc_model.ypred_16, ecc_model.ypred_84, color='grey', alpha=0.2)
ax[2].axvline(np.log(3.5), color='C3', ls=':', lw=2, zorder=0)
ax[2].set_xlim(np.log(0.5), np.log(16))
ax[2].set_xticks(np.log(Rp), Rp)
ax[2].set_ylim(0,0.3)
ax[2].set_ylabel("Eccentricity", fontsize=14)
ax[2].set_xlabel("Planet Radius (Earth-radii)", fontsize=14)


plt.tight_layout()
plt.savefig(os.path.join(PROJECT_DIR, "Figures/3RE-transition.pdf"), bbox_inches='tight')
plt.show()

## Period-Radius diagram

In [ ]:
import seaborn as sns

sns.set_context('paper', font_scale=1.5)

X = pd.DataFrame()
X['lnP'] =np.log10(catalog.period.values)
X['lnR'] = np.log10(catalog.rp.values)

plt.figure(figsize=(6,5))

sns.kdeplot(X, x='lnP', y='lnR', cmap=sns.light_palette("#555",as_cmap=True), fill=True, levels=6)
plt.scatter(X['lnP'], X['lnR'], s=12, facecolors='none', edgecolors='k', lw=0.1, alpha=0.8)

x = np.linspace(np.log10(0.5), np.log10(330), 100)
y = np.log10(1.84*(10**x/10)**-0.11)

plt.plot(x, np.log10(16)*np.ones(100), color='k', lw=1, ls=':')
plt.plot(x, np.log10(8/1.01)*np.ones(100), color='k', lw=1, ls=':')
plt.plot(x, np.log10(4/1.01)*np.ones(100), color='k', lw=1, ls=':')
plt.plot(x, y-np.log10(1.01), color='k', lw=2, ls='--')
plt.plot(x, np.log10(1*1.01)*np.ones(100), color='k', lw=1, ls=':')
plt.plot(x, np.log10(0.5)*np.ones(100), color='k', lw=1, ls=':')

plt.xlim(np.log10(1/1.1), np.log10(100*1.1))
plt.xticks(np.log10([1,3,10,30,100]), [1,3,10,30,100])
plt.ylim(np.log10(0.5/1.1), np.log10(16*1.1))
plt.yticks(np.log10([0.5,1,2,3,4,8,16]), [0.5,1,2,3,4,8,16])
plt.xlabel("Period (days)")
plt.ylabel("Radius ($R_{\oplus}$)")

plt.savefig(os.path.join(PROJECT_DIR, 'Figures/population.pdf'), bbox_inches='tight')
plt.show()

In [ ]:
sns.set_palette?